In [2]:
import pandas as pd
from string import ascii_uppercase
codes = pd.DataFrame()
for letter in ascii_uppercase:
    new_codes = pd.read_html("https://en.wikipedia.org/wiki/List_of_airports_by_IATA_code:_{}".format(letter),header=0)[0]
    if len(new_codes.columns) != 4:
        new_codes.drop(new_codes.columns[4:],inplace=True,axis=1)
        codes = codes.append(new_codes)
    else:
        codes = codes.append(new_codes)

In [ ]:
def find_airport_country(iata):
    temp = iata[0].upper()
    result = codes.loc[codes["IATA"]==iata,"Location served"].str.split(",").str[-1].str.strip().item()
    return result

In [ ]:
def compare_airport_country(dep_country, arr_country):
    if dep_country == arr_country:
        return True
    else:
        return False

In [ ]:
flights = pd.DataFrame(columns=['date','departure_airport','arrival_airport','airline','departure_country','arrival_country'])

In [ ]:
for day in range(1,8):
    formatted_date = (dt.date.today() + dt.timedelta(days=day)).strftime("%Y-%m-%d")
    params = {'access_key':'4666e4d10ce09fe420c4f2518d6286cf',
              'flight_status':'scheduled',
              'flight_date':formatted_date}
    f = requests.get('https://api.aviationstack.com/v1/flights',params)
    if f.json()['pagination']['total']%100==0:
        pages = int(f.json()['pagination']['total']/100)
    else:
        pages = int(f.json()['pagination']['total']/100) + 1
    
    for page in [page * 100 for page in range(pages)]:
        params['offset'] = page
        f = requests.get('https://api.aviationstack.com/v1/flights',params)
        for flight in range(len(f.json()['data'])):
            flight_date = dt.datetime.strptime(f.json()['data'][flight]['flight_date'],"%Y-%m-%d")
            dt.datetime.strptime(f.json()['data'][flight]['flight_date'],"%Y-%m-%d")
            dep_airport = f.json()['data'][flight]['departure']['airport']
            arr_airport = f.json()['data'][flight]['arrival']['airport']
            airline = f.json()['data'][flight]['airline']['name']
            dep_country = find_airport_country(f.json()['data'][flight]['departure']['iata'])
            arr_country = find_airport_country(f.json()['data'][flight]['arrival']['iata'])

            flights = flights.append({'date':flight_date,
                                    'departure_airport':dep_airport,
                                    'arrival_airport':arr_airport,
                                    'airline':airline,
                                    'departure_country':dep_country,
                                    'arrival_country':arr_country} )

In [ ]:
flights['international'] = np.where(flights['departure_country']==flights['arrival_country'],False,True)

In [ ]:
print(flights.departure_country.unique())

In [ ]:
print(codes.iloc[:,-1].str.split(",").str[-1].str.strip().unique())